# Lidando com Dados Ausentes

Por que um conjunto de dados pode apresentar valores ausentes? Às vezes, uma observação não pôde ser registrada por qualquer motivo, como um sensor ou instrumento quebrado ou um respondente de uma pesquisa que optou por não responder a uma pergunta. Infelizmente, o aprendizado de máquina e os modelos estatísticos geralmente não lidam bem com valores ausentes. Por esse motivo, você provavelmente considerará removê-los.

Nesta seção, abordaremos como identificar e remover dados relacionados a valores ausentes.

> Observe que você deve entender completamente por que os valores estão ausentes e rastrear a fonte que produziu os dados. Você também deve estar atento a quaisquer vieses de seleção que possam surgir devido aos dados ausentes. Por exemplo, se você remover registros de respondentes da pesquisa que não responderam a uma determinada pergunta... isso pode enviesar seus modelos em relação à população que optou por responder! Às vezes, é mais interessante perguntar por que os valores estão ausentes do que simplesmente ignorá-los.

Para configurar, vamos acessar um banco de dados SQLite e extrair os dados da tabela `WEATHER_MONITOR`. No entanto, analisaremos apenas uma região de estações, obtendo apenas registros com `LOCATION_ID` de `2`, `28` ou `48`, referentes ao mês de abril. Usaremos o SQL para fazer essa filtragem.

In [ ]:
import urllib.request
import sqlite3
import pandas as pd 

# Baixe o banco de dados SQLite e conecte-se a ele
urllib.request.urlretrieve("https://github.com/thomasnield/machine-learning-demo-data/blob/master/relational/company_operations.db?raw=true", "company_operations.db") 

conn = sqlite3.connect('company_operations.db')

df = pd.read_sql("""
SELECT * FROM WEATHER_MONITOR 
WHERE LOCATION_ID IN (2,28,48) 
AND strftime('%m', REPORT_DATE) = '04'

""", conn, parse_dates=['REPORT_DATE'])
df

> Se quiser aprender mais sobre SQL, confira o [curso Anaconda aqui](https://learning.anaconda.cloud/introduction-to-sql).

Observe acima que a coluna `RAIN` possui valores `NaN`, o que significa que esses valores são `None` e estão ausentes. Há também um registro em que o valor `TEMPERATURE` está ausente. Vamos aprender algumas técnicas sobre como identificar e lidar com eles.

## Rastreamento de valores ausentes

Para encontrar valores ausentes, podemos usar a função `isna()` em um dataframe.

In [ ]:
df.isna()

Também podemos usar a função `any()` para ver quais colunas contêm valores ausentes.

In [ ]:
df.isna().any()

Você também pode inverter o eixo para `any()` e obter uma série booleana informando se cada linha contém um valor ausente.

In [ ]:
df.isna().any(axis=1)

Observe que também existe uma contraparte `notna()` que inverte a condição e define os valores fornecidos como `True` e os valores ausentes como `False`. Há também os aliases `isnull()` e `notnull()`, que são apenas nomes diferentes para as mesmas operações.

É claro que podemos pegar essa série booleana e passá-la para o getter `loc` para recuperar as colunas com valores `NaN`.

In [ ]:
df.loc[:, df.isna().any()]

Para encontrar valores ausentes em colunas específicas, podemos usar uma operação de filtragem com operadores lógicos. Aqui, encontramos todos os registros em que havia um valor `TEMPERATURA` ou `CHUVA` ausente.

In [ ]:
df[df['TEMPERATURE'].isna() | df['RAIN'].isna()]

We can also filter for all records containing any missing values across all fields.

In [ ]:
df[df.isna().any(axis=1)]

## Removendo linhas com valores ausentes

Como mencionado anteriormente, muitos modelos estatísticos e de aprendizado de máquina não toleram `NA`, `NaN` ou outros valores nulos ausentes. Se você entender por que eles estão ausentes e não achar que sua ausência influenciará significativamente seu modelo, poderá simplesmente remover os registros com valores ausentes.

Você pode usar o operador `drop()` com lógica condicional, como aprendemos nas seções anteriores, mas também existe uma função `dropna()` útil para esse propósito.

Abaixo, usamos `dropna()` para remover todos os registros com valores `NaN`. Observe que não estou usando o parâmetro `inplace=True` aqui, portanto, posso demonstrar outros exemplos posteriormente.

In [ ]:
df.dropna(axis=0) # use inplace=True para substituir o dataframe atual

Observe que os quatro registros com valores `NaN` para `RAIN` ou `TEMPERATURE` foram removidos. Também podemos fornecer apenas um `subconjunto` de índices a serem considerados para a remoção de valores nulos. Abaixo, removemos apenas os registros onde `NA` existe na coluna `RAIN`.

In [ ]:
df.dropna(axis=0, subset=["RAIN"])

Se preferirmos descartar as colunas com valores `NaN`, podemos usar `axis=1`.

In [ ]:
df.dropna(axis=1)

## Substituindo Valores Ausentes

Embora isso possa não fazer sentido do ponto de vista do aprendizado de máquina, pode haver momentos em que você queira substituir valores ausentes. Você pode fazer isso usando a função `fillna()`. Abaixo, substituímos todos os valores `na` em nossa tabela por `-1`. Infelizmente, não há um parâmetro `subset` para esta função, portanto, para atingir colunas específicas, você precisará extraí-las, aplicar a função `fillna()` e, em seguida, atribuí-las de volta.

In [ ]:
df.fillna(value=-1, inplace=True)
df

> Existem outros métodos que `fillna()` pode usar para preencher valores ausentes. [Leia a documentação do Pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna) para saber mais.

Por outro lado, pode haver momentos em que você queira substituir certos valores por `na`, como strings vazias ou strings de espaço reservado como `NULL`. Acabamos de transformar os valores `NaN` em `-1`. Vamos convertê-los de volta para `NaN` usando a função `replace()`.

In [ ]:
from numpy import nan

df.replace(-1, nan, inplace=True)
df

## Preencha os valores ausentes com a média

Outra maneira de lidar com valores ausentes que pode ser mais adequada para aprendizado de máquina e modelos estatísticos é usar uma substituição de valor estatístico, como uma `mean` ou `median`.

Vamos usar o `SimpleImputer` do scikit-learn e configurá-lo para usar a `média`.

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

Vamos então aplicar os campos `TEMPERATURE` e `RAIN` ao imputador.

In [ ]:
transform_input = df[['TEMPERATURE','RAIN']]

imputer.fit(transform_input)

O `SimpleImputer` calculará a média para `RAIN` como `0.8585` e para `TEMPERATURE` como `59.740909`. Podemos então aplicar essas colunas com as médias substituindo os `NaN`s ao nosso dataframe.

In [ ]:
# copia o dataframe do clima
mean_df = df.copy() 

# aplica média às colunas TEMPERATURE e RAIN
transform_output = imputer.transform(transform_input)
mean_df[['TEMPERATURE','RAIN']] = transform_output
mean_df

Observe que há outras opções para o parâmetro `strategy`, incluindo 'mean', 'median', 'most_frequent' e 'constant'.

## Preencha os valores ausentes com o vizinho mais próximo

Outra opção para imputar um valor para substituir valores ausentes é utilizar o k-vizinho mais próximo (KNN), que funciona muito bem em muitos casos. Essencialmente, a ideia é encontrar pontos de dados próximos daquele com o valor ausente, considerando todos os campos. Esses registros vizinhos são então usados ​​para inferir uma estimativa para o valor ausente.

Vamos usar o `KNNInputer` e os 5 vizinhos mais próximos. Uniformizaremos os pesos e informaremos para não ignorar valores `NaN`, definindo `metric` como `nan_euclidean`.

In [ ]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

Como estamos inferindo com base em outros campos, precisamos remover `ID`, `LOCATION_ID` e `REPORT_CODE`, pois não são úteis para o modelo KNN. São valores arbitrários ou gerados aleatoriamente e não têm valor preditivo. Também precisamos converter `REPORT_DATE` para um valor numérico. Felizmente, estamos trabalhando apenas com um mês, então vamos pegar apenas o dia do mês e essa será nossa conversão numérica.

In [ ]:
# copia o dataframe e remova duas colunas que não são úteis para modelagem
knn_input = df.drop(['ID','REPORT_CODE','LOCATION_ID'],axis=1)

# extrai o dia do mês e torne-o `REPORT_DATE`
knn_input['REPORT_DATE'] = knn_input['REPORT_DATE'].dt.strftime('%d').astype(int)

# ajusta o modelo knn
imputer.fit(knn_input)
knn_input

Por fim, vamos copiar o dataframe e aplicar a transformação às duas colunas. Os valores `RAIN` com índices de linha 5, 18 e 19 estavam ausentes, mas agora são `0.604`, `0.996` e `0.986`, respectivamente. O valor `TEMPERATURE` ausente do índice de linha 12 é inferido como `55.76`.

In [ ]:
# copie o dataframe
knn_output = df.copy()

# aplica transformação knn à entrada
knn_transform = imputer.transform(knn_input)

# aplica somente as colunas TEMPERATURA e CHUVA de volta ao dataframe
knn_output.loc[:,["TEMPERATURE","RAIN"]] = knn_transform[:,[1,3]]
knn_output

## Exercício

O código abaixo contém um exemplo de dados do termostato. Complete o código abaixo para que a mediana seja imputada para os valores ausentes dos campos `temperatura` e `umidade`.

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, None, 57.2, 57.4, None, 27.5], 
    "humidity" : [.8, None, .7, .6, .7, .7, .8]
})

# criar e ajustar imputador
imputer = ?
transform_input = df[['temperature','humidity']]
imputer.fit(?)

# aplicar média à coluna de temperatura e umidade
transform_output = imputer.transform(?)
df[['temperature','humidity']] = transform_output
df

### RESPOSTA A BAIXO

|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
|<br>
v 

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer

df = pd.DataFrame({
    "record_id" : ['OVUTJE','OVUTJE','WI4QEX','WI4QEX','FS40NF','O64LIT','U888EA'],
    "temperature" : [65.2, 65.2, None, 57.2, 57.4, None, 27.5], 
    "humidity" : [.8, None, .7, .6, .7, .7, .8]
})

# criar e ajustar imputador
imputer = SimpleImputer(strategy='median')
transform_input = df[['temperature','humidity']]
imputer.fit(transform_input)

# aplicar média às colunas TEMPERATURE e RAIN
transform_output = imputer.transform(transform_input)
df[['temperature','humidity']] = transform_output
df